In [1]:
import os
import json
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [2]:
def load_data(directory="Data to test the model"):
    required = {"open","high","low","close","volume","turnover"}
    data = {}
    for fname in os.listdir(directory):
        if not fname.endswith("_15m_test.csv"):
            continue
        sym = fname.replace("_15m_test.csv", "")
        path = os.path.join(directory, fname)
        try:
            df = (
                pd.read_csv(path, parse_dates=["timestamp"])
                .set_index("timestamp")
            )
            if required.issubset(df.columns):
                data[sym] = df
        except Exception as e:
            print(f"Error loading {fname}: {e}")
    if not data:
        raise RuntimeError(f"No valid data in {directory}")
    return data

In [3]:
class SampleDataset(Dataset):
    def __init__(self, data_dir, target, L, H,
                 features=("open","high","low","close","volume","turnover")):
        raw = load_data(data_dir)
        syms = sorted(raw)
        if target not in syms:
            raise ValueError(f"{target} not found in {syms}")
        self.feat = list(features)
        # חתוך לכל הסימולים לאורך המינימלי
        min_T = min(df.shape[0] for df in raw.values())
        arr = torch.tensor(
            [ raw[s][self.feat].values[-min_T:] for s in syms ],
            dtype=torch.float32
        )  # shape=(S, F, T)
        self.data = arr
        self.S, self.F, self.T = arr.shape
        self.target_idx = syms.index(target)
        self.close_idx  = self.feat.index("close")
        self.L, self.H = L, H
        self.n = max(0, self.T - L - H + 1)

    def __len__(self):
        return self.n

    def __getitem__(self, i):
        x = self.data[:, :, i : i + self.L]           # [S, F, L]
        seq = self.data[
            self.target_idx, self.close_idx,
            i + self.L : i + self.L + self.H
        ]
        # ממיר לתשואה לוגריתמית עם אפס בתחילה
        prev, curr = seq[:-1], seq[1:]
        ret = torch.log(curr / (prev + 1e-6))
        y = torch.cat([torch.zeros(1), ret])         # length H
        return x, y


In [4]:
class ConvTCNModel(nn.Module):
    def __init__(self, F, L, H):
        super().__init__()
        self.extractor = nn.Sequential(
            nn.Conv1d(F, 32, 3, stride=2, padding=1), nn.ELU(),
            nn.BatchNorm1d(32),
            nn.Conv1d(32, 64, 3, stride=2, padding=1), nn.ELU(),
            nn.BatchNorm1d(64),
        )
        self.seq_len = L // 4
        self.tcn = nn.Sequential(
            nn.Conv1d(64, 64, 3, padding=1), nn.ELU(), nn.BatchNorm1d(64),
            nn.Conv1d(64, 64, 3, padding=1), nn.ELU(), nn.BatchNorm1d(64),
        )
        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Sequential(
            nn.Linear(64, 128), nn.ELU(), nn.LayerNorm(128),
            nn.Linear(128, H),
        )

    def forward(self, x):
        B, S, F, L = x.shape
        z = x.view(B * S, F, L)
        z = self.extractor(z)
        z = self.tcn(z)
        z = z.view(B, S, 64, self.seq_len).mean(dim=1)
        z = self.pool(z).squeeze(-1)
        return self.fc(z)


In [ ]:
if __name__ == "__main__":
    # טען פרמטרים
    cfg = json.load(open("BTC_model/shape_params.json"))
    F, L, H = cfg["F"], cfg["L"], cfg["future_steps"]

    # הכנת דאטה
    ds = SampleDataset("Data to test the model", "BTCUSDT", L, H)
    loader = DataLoader(ds, batch_size=32, shuffle=False)

    # טען מודל
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = ConvTCNModel(F, L, H).to(device)
    state = torch.load("BTC_model/model_BTC.pth", map_location=device)
    model.load_state_dict(state)
    model.eval()

    # הערכה
    mse, total, count = nn.MSELoss(), 0.0, 0
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            preds = model(x)
            total += mse(preds, y).item() * x.size(0)
            count += x.size(0)

    if count:
        print("Test MSE =", total / count)
    else:
        print("אין דגימות לבחינה (n=0). בדוק sample_len ו-future_steps לאורך הנתונים.")



C:\Users\GameReaper\AppData\Local\Temp\ipykernel_18060\3731102922.py:11: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:278.)
  arr = torch.tensor(


אין דגימות לבחינה (n=0). בדוק sample_len ו-future_steps לאורך הנתונים.


In [6]:
import os
print(os.listdir("Data to test the model"))


['BTCUSDT_15m_test.csv', 'LTCUSDT_15m_test.csv', 'LUNA2USDT_15m_test.csv', 'MAGICUSDT_15m_test.csv', 'MANAUSDT_15m_test.csv', 'MASKUSDT_15m_test.csv', 'MAVUSDT_15m_test.csv', 'MBLUSDT_15m_test.csv', 'MDTUSDT_15m_test.csv', 'MINAUSDT_15m_test.csv', 'MKRUSDT_15m_test.csv', 'MTLUSDT_15m_test.csv', 'NEARUSDT_15m_test.csv', 'NKNUSDT_15m_test.csv', 'NMRUSDT_15m_test.csv', 'OGNUSDT_15m_test.csv', 'OGUSDT_15m_test.csv', 'OMGUSDT_15m_test.csv', 'ONEUSDT_15m_test.csv', 'ONTUSDT_15m_test.csv', 'OPUSDT_15m_test.csv', 'ORBSUSDT_15m_test.csv', 'ORDIUSDT_15m_test.csv', 'OXTUSDT_15m_test.csv', 'PAXGUSDT_15m_test.csv', 'PENDLEUSDT_15m_test.csv', 'PEOPLEUSDT_15m_test.csv', 'PERPUSDT_15m_test.csv', 'PHBUSDT_15m_test.csv', 'QNTUSDT_15m_test.csv', 'QTUMUSDT_15m_test.csv', 'RADUSDT_15m_test.csv', 'RDNTUSDT_15m_test.csv', 'REQUSDT_15m_test.csv', 'RLCUSDT_15m_test.csv', 'ROSEUSDT_15m_test.csv', 'RPLUSDT_15m_test.csv', 'RSRUSDT_15m_test.csv', 'RSS3USDT_15m_test.csv', 'RUNEUSDT_15m_test.csv', 'RVNUSDT_15m_test.